In [1]:
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.3/268.3 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.3 MB/s eta 0:00:00


In [10]:
!pip install python-dotenv

In [16]:
import os
import re
import json
from openai import AzureOpenAI
from typing import Optional

class GPTModel:
  def __init__(self, api_key: str, endpoint: str, model: str):
    self.client = AzureOpenAI(
      azure_endpoint = endpoint,
      api_key = api_key,
      api_version = "2024-02-01"
    )
    self.model = model

  @staticmethod
  def extract_json_content(s: str) -> str:
    # Define the regex pattern to match `json<content>`
    pattern = r'^```json(.*)```$'

    # Search for the pattern
    match = re.search(pattern, s, re.DOTALL)

    # If a match is found, return the content within the wrapper
    if match:
        return match.group(1).strip()
    else:
        # Return the original string or handle as needed
        return s

  @staticmethod
  def json_to_dict(s: str) -> dict:
    return json.loads(s)

  def call(self, user_msg: str, sys_msg: Optional[str] = None) -> str:
    messages = []

    if sys_msg:
      messages.append({"role": "system", "content": sys_msg})
    messages.append({"role": "user", "content": user_msg})

    response = self.client.chat.completions.create(
        model=self.model,
        messages=messages,
    )

    res = response.choices[0].message.content
    res = GPTModel.extract_json_content(res)
    res = GPTModel.json_to_dict(res)
    return res

In [19]:
!cat .env

AZURE_ENDPOINT=https://ihub-astd-brunet-openai.openai.azure.com/
AZURE_API_KEY=5f4307496a79407b802eb5c8a2cc60c0
AZURE_MODEL=ihub-astd-brunet-gpt4-32k

In [17]:
from dotenv import load_dotenv
load_dotenv()

AZURE_ENDPOINT = os.getenv('AZURE_ENDPOINT')
AZURE_API_KEY = os.getenv('AZURE_API_KEY')
AZURE_MODEL = os.getenv('AZURE_MODEL')

In [20]:
import pydantic

from pydantic import BaseModel

class User(BaseModel):
    name: str
    age: int

user = User(name='John Doe', age=30)

sys_msg = f"Your response to the user's request should follow the following JSON structure: {user.json()}"
user_msg = f"Can you come up with an arbituaray person with a name and an age?"

gpt_model = GPTModel(
    api_key=AZURE_API_KEY,
    endpoint=AZURE_ENDPOINT,
    model=AZURE_MODEL
)

res = gpt_model.call(user_msg, sys_msg)
print(res)

{'name': 'William Smith', 'age': 45}
